In [ ]:
import sys
import mysql.connector
import xlsxwriter
import pandas as pd
import sys
import mysql.connector
import os
from dotenv import load_dotenv
load_dotenv()
#ESTE ARGUMENTO NO SE USA EN ESTE REPORTE, SERÁ 0 SIEMPRE UWU
id=0
#configurar la conexion a la base de datos
DB_USERNAME = os.getenv('DB_USERNAME')
DB_DATABASE = os.getenv('DB_DATABASE')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_PORT = os.getenv('DB_PORT')

a_color='#354F84'
b_color='#91959E'
# Conectar a DB
cnx = mysql.connector.connect(user=DB_USERNAME,
                              password=DB_PASSWORD,
                              host='localhost',
                              port=DB_PORT,
                              database=DB_DATABASE,
                              use_pure=False)
#Seccion para traer informacion de la base
query = ('SELECT * from customers where id =1')

# join para cobros
# cobros=pd.read_sql('Select cobros.* ,customers.customer,internal_orders.invoice, users.name from ((cobros inner join internal_orders on internal_orders.id = cobros.order_id) inner join customers on customers.id = internal_orders.customer_id )inner join users on cobros.capturo=users.id',cnx)


#traer datos de los pedidos
pedidos=pd.read_sql("""Select internal_orders.* ,customers.clave,customers.alias,
coins.exchange_sell, coins.coin, coins.symbol,coins.code
from ((
    internal_orders
    inner join customers on customers.id = internal_orders.customer_id )
    inner join coins on internal_orders.coin_id = coins.id)
                    where status  != 'CANCELADO'
     """,cnx)
clientes=pd.read_sql("select * from customers",cnx)
cobros=pd.read_sql("""select cobros.*,internal_orders.coin_id as coin_pedido,internal_orders.customer_id  
                     from (
                         cobros
    inner join internal_orders on internal_orders.id = cobros.order_id )""",cnx)
facturas=pd.read_sql("""select factures.*,internal_orders.coin_id as coin_pedido,internal_orders.customer_id  
                     from (
                         factures
    inner join internal_orders on internal_orders.id = factures.order_id )""",cnx)
creditos=pd.read_sql("""select credit_notes.*,internal_orders.coin_id as coin_pedido,internal_orders.customer_id as customer_pedido 
                     from (
                         credit_notes
    inner join internal_orders on internal_orders.id = credit_notes.order_id ) """,cnx)

cobros2=pd.read_sql("""select cobro_orders.*,internal_orders.coin_id as coin_pedido,internal_orders.customer_id 
                     from (((
                         cobro_orders 
    inner join cobros on cobros.id=cobro_orders.cobro_id)
    inner join internal_orders on internal_orders.id = cobros.order_id )
    inner join coins on internal_orders.coin_id = coins.id) """,cnx)

nordenes=len(pedidos)
df=pedidos[['date']]

#Filtrando datos---------------------------------------
#Separar pedidos vigentes
    #Agregar columna con saldo en los cobros
pedidos=pedidos.assign(saldo=0.0)
for i in range(len(pedidos)):
    pedidos['saldo'].values[i]=cobros2.loc[cobros2['order_id']==pedidos['id'].values[i],'amount'].sum()
pedidos=pedidos.loc[pedidos['total']-pedidos['saldo']>1]
cobros2=cobros2.merge(pedidos[['id','code']].rename(columns={'id':'order_id'}),how='inner',on='order_id')

tc=pd.read_sql('select * from coins where id=13 ',cnx)['exchange_sell'].values[0]

writer = pd.ExcelWriter('storage/report/CxC_cliente_consolidado_vigente'+str(id)+'.xlsx', engine='xlsxwriter')
workbook = writer.book
##FORMATOS PARA EL TITULO------------------------------------------------------------------------------
rojo_l = workbook.add_format({
    'bold': 0,
    'border': 0,
    'align': 'center',
    'valign': 'vcenter',
    #'fg_color': 'yellow',
    'font_color': 'red',
    'font_size':16})
negro_s = workbook.add_format({
    'bold': 0,
    'border': 0,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    'font_size':12})
negro_b = workbook.add_format({
    'bold': 2,
    'border': 0,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    'font_size':13,
    
    'text_wrap': True,
    'num_format': 'dd/mm/yyyy'}) 
rojo_b = workbook.add_format({
    'bold': 2,
    'border': 0,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'red',
    'font_size':13})      

#FORMATOS PARA CABECERAS DE TABLA --------------------------------
header_format = workbook.add_format({
    'bold': True,
    'text_wrap': True,
    'valign': 'center',
    'fg_color': 'yellow',
    'border': 1,})

blue_header_format = workbook.add_format({
    'bold': True,
    'bg_color': a_color,
    'text_wrap': True,
    'valign': 'vcenter',
    'align': 'center',
    'border_color':'white',
    'font_color': 'white',
    'num_format': '[$$-409]#,##0.00',
    'border': 1})
blue_header_format_bold = workbook.add_format({
    'bold': True,
    'bg_color': a_color,
    'text_wrap': True,
    'valign': 'top',
    'align': 'center',
    'border_color':'white',
    'font_color': 'white',
    'num_format': '[$$-409]#,##0.00',
    'border': 1,
    'font_size':13})
blue_footer_format_bold = workbook.add_format({
    'bold': True,
    'bg_color': a_color,
    'text_wrap': True,
    'valign': 'top',
    'align': 'center',
    'border_color':'white',
    'font_color': 'white',
    'border': 1,
    'num_format': '[$$-409]#,##0.00',
    'font_size':11})
red_header_format = workbook.add_format({
    'bold': True,
    'bg_color': b_color,
    'text_wrap': True,
    'valign': 'top',
    'align': 'center',
    'border_color':'white',
    'font_color': 'white',
    'border': 1})

red_header_format_bold = workbook.add_format({
    'bold': True,
    'bg_color': b_color,
    'text_wrap': True,
    'valign': 'top',
    'align': 'center',
    'border_color':'white',
    'font_color': 'white',
    'border': 1,
    'font_size':13})


#FORMATOS PARA TABLAS PER CE------------------------------------

blue_content = workbook.add_format({
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    
    'border_color':a_color,
    'font_size':10,
    'num_format': '[$$-409]#,##0.00'})

blue_content_bold = workbook.add_format({
    'bold': True,
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    'font_size':11,
    'border_color':a_color,
    'num_format': '[$$-409]#,##0.00'})

blue_content_bold_dll = workbook.add_format({
    'bold': True,
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    'font_size':11,
    'bg_color': '#b4e3b1',
    'border_color':a_color,
    'num_format': '[$$-409]#,##0.00'})
blue_content_footer_dll = workbook.add_format({
    'bold': True,
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'white',
    'font_size':11,
    'bg_color': '#356e31',
    'border_color':'white',
    'num_format': '[$$-409]#,##0.00'})
blue_content_footer = workbook.add_format({
    'bold': True,
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'white',
    'font_size':11,
    'bg_color': '#3e5585',
    'border_color':'white',
    'num_format': '[$$-409]#,##0.00'})
blue_content_date = workbook.add_format({
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    'font_size':9,
    'border_color':a_color,
    'num_format': 'dd/mm/yyyy'})
#FOOTER FORMATS---------------------------------------------------------
observaciones_format = workbook.add_format({
    'bold': True,
    'text_wrap': True,
    'valign': 'top',
    'fg_color':'#BDD7EE',
    'border': 1})

blue_content_dll = workbook.add_format({
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    'bg_color': '#b4e3b1',
    'border_color':a_color,
    'font_size':10,
    'num_format': '[$$-409]#,##0.00'})
total_cereza_format = workbook.add_format({
    'bold': True,
    'text_wrap': True,
    'valign': 'top',
    'fg_color':'#F4B084',
    'border': 1})


df[0:1].to_excel(writer, sheet_name='Sheet1', startrow=7,startcol=6, header=False, index=False)
worksheet = writer.sheets['Sheet1']
#Encabezado del documento--------------

import datetime

currentDateTime = datetime.datetime.now()
date = currentDateTime.date()
year = date.strftime("%Y")
df[0:1].to_excel(writer, sheet_name='Sheet1', startrow=7,startcol=6, header=False, index=False)
worksheet = writer.sheets['Sheet1']
#Encabezado del documento--------------
worksheet.merge_range('B2:F2', 'CUENTAS POR COBRAR REPORTE 2B ', negro_b)
worksheet.merge_range('B3:F3', 'DERECHOS ADQUIRIDOS POR COBRAR VIGENTES', negro_s)
worksheet.merge_range('B4:F4', 'CLASIFICADAS POR CLIENTE CONSOLIDADO', negro_b)

worksheet.write('H2', 'AÑO', negro_b)

worksheet.write('I2', year, negro_b)
worksheet.merge_range('J2:K3', """FECHA DEL REPORTE             
DD/MM/AAAA""", negro_b)

worksheet.merge_range('L2:L3', date, negro_b)
worksheet.insert_image("A1", "img/logo/logo.png",{"x_scale": 0.6, "y_scale": 0.6})
worksheet.merge_range('C6:C10', """PDA""", blue_header_format)
worksheet.merge_range('D6:D10', """PI CANTIDAD""", blue_header_format)
worksheet.merge_range('E6:E10', """FECHA
DD-MM-AAAA""", blue_header_format)

worksheet.merge_range('F6:G9', 'CLIENTE', blue_header_format)
worksheet.write('F10', 'NUMERO', blue_header_format)
worksheet.write('G10', 'NOMBRE CORTO', blue_header_format)

worksheet.merge_range('H6:H10', """MONEDA""", blue_header_format)

worksheet.merge_range('I6:O6', 'DERECHOS ADQUIRIDOS', blue_header_format)
worksheet.merge_range('I7:J9', """IMPORTE TOTAL 
(DERECHOS ADQUIRIDOS) 
SIN IVA""", blue_header_format)
worksheet.write('I10', 'MXN', blue_header_format)
worksheet.write('J10', 'DLLS', blue_header_format)


worksheet.merge_range('K7:L9', """COBRADO
(IMPORTE TOTAL COBRADO)
SIN IVA""", blue_header_format)
worksheet.write('K10', 'MXN', blue_header_format)
worksheet.write('L10', 'DLLS', blue_header_format)


worksheet.merge_range('M7:N9', """POR COBRAR
(IMPORTE TOTAL POR COBRAR) 
SIN IVA""", blue_header_format)
worksheet.write('M10', 'MXN', blue_header_format)
worksheet.write('N10', 'DLLS', blue_header_format)


worksheet.merge_range('O7:O10', '% POR COBRAR DEL PEDIDO INTERNO', blue_header_format)

worksheet.merge_range('P6:S6', """DERECHOS ADQUIRIDOS POR COBRAR CONTABLES""", blue_header_format)
worksheet.merge_range('P7:Q9', """FACTURADO
CxC CONTABLES
(SIN IVA)""", blue_header_format)
worksheet.write('P10', 'MXN', blue_header_format)
worksheet.write('Q10', 'DLLS', blue_header_format)


worksheet.merge_range('R7:S9', """POR FACTURAR
DA X C
(SIN IVA)""", blue_header_format)
worksheet.write('R10', 'MXN', blue_header_format)
worksheet.write('S10', 'DLLS', blue_header_format)

worksheet.merge_range('T6:T10', """

ESTATUS""", blue_header_format)

counter=0
total_total=0
pedidos['code']=pedidos['code'].str.replace('MN','MXN')
pedidos['reg_date']=pd.to_datetime(pedidos['reg_date'], format='%Y-%m-%d')
pedidos['reg_date']=pedidos['reg_date'].dt.strftime('%d-%m-%Y')
pedidos.loc[pedidos['status'] == 'CANCELADO','total']= 0

clientes=clientes.sort_values(by='clave')
for i in range(0,len(clientes)):
   if(len(pedidos.loc[pedidos['customer_id']==clientes['id'].values[i]])>0):
        
        print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        print(str(i)+'cliente',str(clientes['clave'].values[i]),clientes['alias'].values[i])
        row_index=str(11+counter)
        counter=counter+1
        cobros_mn=cobros2.loc[(cobros2['customer_id']==clientes['id'].values[i])&(cobros2['coin_pedido']==1)]
        facturas_mn=facturas.loc[(facturas['customer_id']==clientes['id'].values[i])&(facturas['coin_pedido']==1)]
        notas_mn=creditos.loc[(creditos['customer_pedido']==clientes['id'].values[i])&(creditos['coin_pedido']==1)]
        
        cobros_dlls=cobros2.loc[(cobros2['customer_id']==clientes['id'].values[i])&(cobros2['coin_pedido']!=1)]
        facturas_dlls=facturas.loc[(facturas['customer_id']==clientes['id'].values[i])&(facturas['coin_pedido']!=1)]
        notas_dlls=creditos.loc[(creditos['customer_id']==clientes['id'].values[i])&(creditos['coin_pedido']!=1)]
        pedidos_mn=pedidos.loc[(pedidos['customer_id']==clientes['id'].values[i])&(pedidos['coin_id']==1)]
        pedidos_dlls=pedidos.loc[(pedidos['customer_id']==clientes['id'].values[i])&(pedidos['coin_id']!=1)]
        
       
        total_mn=  (pedidos_mn['total']/1.16).sum()
        total_dlls=pedidos_dlls['total'].sum()/1.16
        total_total=total_total+total_mn+total_dlls*tc
        #datos generales del pedido
        #worksheet.write('B'+row_index, str(pedidos['noha'].values[i]), blue_content)
        worksheet.write('C'+row_index, str(i+1), blue_content)
        worksheet.write('D'+row_index, str(len(pedidos.loc[pedidos['customer_id']==clientes['id'].values[i]])), blue_content)
        worksheet.write('E'+row_index, str(pedidos.loc[pedidos['customer_id']==clientes['id'].values[i],'reg_date'].values[0]), blue_content_date)
        worksheet.write('F'+row_index, str(clientes['clave'].values[i]).replace(" ",""), blue_content)
        worksheet.write('G'+row_index, str(clientes['alias'].values[i]), blue_content)
        worksheet.write('H'+row_index, str(pedidos.loc[pedidos['customer_id']==clientes['id'].values[i],'code'].values[0]), blue_content)
        #subtotal
    
        worksheet.write('I'+row_index,total_mn, blue_content)
        worksheet.write('J'+row_index, total_dlls, blue_content_dll)
        
        #cobrado
        worksheet.write('K'+row_index, (cobros_mn['amount']/1.16).sum() , blue_content)
        worksheet.write('L'+row_index,cobros_dlls['amount'].sum()/1.16  , blue_content_dll)
       
        xcobrarmn=max(0,total_mn-cobros_mn['amount'].sum()/1.16) 
        xcobrardll=max(0,total_dlls-cobros_dlls['amount'].sum()/1.16)
        #por cobrar
        worksheet.write('M'+row_index, total_mn-cobros_mn['amount'].sum()/1.16 , blue_content)
        worksheet.write('N'+row_index, total_dlls-cobros_dlls['amount'].sum()/1.16  , blue_content_dll)
    
        worksheet.write('O'+row_index, "{:.2f}".format((total_mn+total_dlls-(cobros_dlls['amount'].sum()+cobros_mn['amount'].sum())/1.16 )*100/(total_dlls+total_mn))+"%", blue_content)
        #facturado

        #calculooooooo
        fwritemn=0
        xfwritemn=0
        fwritedll=0
        xfwritedll=0
        estospedidos=pedidos.loc[pedidos['customer_id']==clientes['id'].values[i]]
        for j in range(len(estospedidos)):
            if(estospedidos['coin_id'].values[j]==1):
                if(estospedidos['total'].values[j]- cobros2.loc[cobros2['order_id']==estospedidos['id'].values[j],'amount'].sum()<=0):
                    fwritemn=fwritemn+0
                else:
                    fwritemn=fwritemn+(facturas.loc[facturas['order_id']==estospedidos['id'].values[j],'amount'].sum()/1.16-creditos.loc[creditos['order_id']==estospedidos['id'].values[j],'amount'].sum()/1.16)-cobros2.loc[cobros2['order_id']==estospedidos['id'].values[j],'amount'].sum()/1.16
                fwritedll=fwritedll+0
                xfwritedll=xfwritedll+0
                xfwritemn=xfwritemn+max(0,(estospedidos['total'].values[j]/1.16-facturas.loc[facturas['order_id']==estospedidos['id'].values[j],'amount'].sum()/1.16+creditos.loc[creditos['order_id']==estospedidos['id'].values[j],'amount'].sum()/1.16))
            else:
                if(estospedidos['total'].values[j]- cobros2.loc[cobros2['order_id']==estospedidos['id'].values[j],'amount'].sum()<=0):
                    fwritedll=fwritedll+0
                else:
                    fwritedll=+fwritedll+(facturas.loc[facturas['order_id']==estospedidos['id'].values[j],'amount'].sum()/1.16-creditos.loc[creditos['order_id']==estospedidos['id'].values[j],'amount'].sum()/1.16)-cobros2.loc[cobros2['order_id']==estospedidos['id'].values[j],'amount'].sum()/1.16
                fwritemn=fwritemn+0
                xfwritemn=xfwritemn+0
                xfwritedll=xfwritedll+max(0,(estospedidos['total'].values[j]/1.16-facturas.loc[facturas['order_id']==estospedidos['id'].values[j],'amount'].sum()/1.16+creditos.loc[creditos['order_id']==estospedidos['id'].values[j],'amount'].sum()/1.16))
        xfwritemn=xcobrarmn- fwritemn
        xfwritedll=xcobrardll- fwritedll
        worksheet.write('P'+row_index,fwritemn, blue_content)
        worksheet.write('Q'+row_index, fwritedll , blue_content_dll)
        

        #por facturar
        worksheet.write('R'+row_index,xfwritemn, blue_content)
        worksheet.write('S'+row_index,xfwritedll  , blue_content_dll)
      #status
        print('totalmn',total_mn)
        print('xcobrar',xcobrarmn)
        print('totaldll',total_dlls)
        print('xcobrardll',xcobrardll)
        if((xcobrarmn>1)|(xcobrardll>1)):
            worksheet.write('T'+row_index,'ACTIVO', blue_content)
        else:
            worksheet.write('T'+row_index,'CERRADO', blue_content)
        
trow=11+counter

worksheet.merge_range('G'+str(trow)+':H'+str(trow), 'Subtotales', blue_header_format_bold)
#SUBTOTAL PEDIDOS MN
worksheet.write('I'+str(trow), pedidos.loc[pedidos['coin_id']==1,'total'].sum()/1.16, blue_content_footer)
#SUBTOTAL PEDIDOS DLLS
worksheet.write('J'+str(trow), pedidos.loc[pedidos['coin_id']!=1,'total'].sum()/1.16, blue_content_footer_dll)
#TOTAL COBRADO MN
worksheet.write_formula('K'+str(trow),  '{=SUM(K9:K'+str(trow-1)+')}', blue_content_footer)
#TOTAL COBRADO DLLS
worksheet.write_formula('L'+str(trow),  '{=SUM(L9:L'+str(trow-1)+')}', blue_content_footer_dll)
#TOTAL POR COBRAR MN
worksheet.write_formula('M'+str(trow),  '{=SUM(M9:M'+str(trow-1)+')}', blue_content_footer)
#TOTAL POR COBRAR DLLS
worksheet.write_formula('N'+str(trow),  '{=SUM(N9:N'+str(trow-1)+')}',blue_content_footer_dll)
#TOTAL FACTURADO MN
#TOTAL FACTURADOR DLLS
worksheet.write_formula('P'+str(trow),  '{=SUM(P9:P'+str(trow-1)+')}', blue_content_footer)
#TOTAL POR FACTURAR MN
worksheet.write_formula('Q'+str(trow),  '{=SUM(Q9:Q'+str(trow-1)+')}', blue_content_footer_dll)
#TOTAL POR FACTURAR DLLS
worksheet.write_formula('R'+str(trow),  '{=SUM(R9:R'+str(trow-1)+')}',blue_content_footer)
worksheet.write_formula('S'+str(trow),  '{=SUM(S9:S'+str(trow-1)+')}',blue_content_footer_dll)



#TIPO DE CAMBIO
worksheet.merge_range('C'+str(trow)+':D'+str(trow),'TIPO DE CAMBIO',blue_header_format)
worksheet.merge_range('C'+str(trow+1)+':D'+str(trow+1),float(tc),blue_header_format)
worksheet.merge_range('C'+str(trow+2)+':D'+str(trow+2),' ',blue_header_format)

#TOTALES
worksheet.merge_range('G'+str(trow+1)+':H'+str(trow+1), 'TOTAL (EQV M.N)', blue_header_format_bold)
worksheet.merge_range('G'+str(trow+2)+':H'+str(trow+2), 'GRAN TOTAL', blue_header_format)
worksheet.merge_range('I'+str(trow+1)+':J'+str(trow+2),' ',blue_footer_format_bold)
worksheet.write_formula('I'+str(trow+1)+':J'+str(trow+2),  '{=(I'+str(trow)+'+J'+str(trow)+' * '+str(tc)+')}',blue_footer_format_bold)


worksheet.merge_range('K'+str(trow+1)+':L'+str(trow+2),' ',blue_footer_format_bold)
worksheet.write_formula('K'+str(trow+1)+':L'+str(trow+2),  '{=(K'+str(trow)+'+L'+str(trow)+' * '+str(tc)+')}',blue_footer_format_bold)

worksheet.merge_range('M'+str(trow+1)+':N'+str(trow+2),' ',blue_footer_format_bold)
worksheet.write_formula('M'+str(trow+1)+':N'+str(trow+2),  '{=(M'+str(trow)+'+N'+str(trow)+' * '+str(tc)+')}',blue_footer_format_bold)

worksheet.merge_range('P'+str(trow+1)+':Q'+str(trow+1),' ',blue_footer_format_bold)
worksheet.write_formula('P'+str(trow+1)+':Q'+str(trow+1),  '{=(P'+str(trow)+'+Q'+str(trow)+' * '+str(tc)+')}',blue_footer_format_bold)


worksheet.merge_range('R'+str(trow+1)+':S'+str(trow+1),' ',blue_footer_format_bold)
worksheet.write_formula('R'+str(trow+1)+':S'+str(trow+1),  '{=(R'+str(trow)+'+S'+str(trow)+' * '+str(tc)+')}',blue_footer_format_bold)

worksheet.merge_range('P'+str(trow+2)+':S'+str(trow+2),' ', blue_footer_format_bold)
worksheet.write_formula('P'+str(trow+2)+':S'+str(trow+2),'{=(P'+str(trow)+'+Q'+str(trow)+' * '+str(tc)+'+R'+str(trow)+'+S'+str(trow)+' * '+str(tc)+')}', blue_footer_format_bold)



# worksheet.write('K'+str(trow), str(cobros['amount'].sum()), blue_content)
# worksheet.write('L'+str(trow), str(cobros['exchange_sell'].values[0]*cobros['amount'].sum()), blue_content_bold)


#RESUMEN
worksheet.merge_range('C'+str(trow+3)+':H'+str(trow+3),'RESUMEN DEL REPORTE',blue_header_format_bold)

worksheet.merge_range('C'+str(trow+4)+':F'+str(trow+4),'DERECHOS ADQUIRIDOS',blue_header_format)
worksheet.merge_range('C'+str(trow+5)+':F'+str(trow+5),'COBRADOS',blue_header_format)
worksheet.merge_range('C'+str(trow+6)+':F'+str(trow+6),'POR COBRAR',blue_header_format)
worksheet.merge_range('C'+str(trow+7)+':F'+str(trow+7),'PEDIDOS REPORTADOS',blue_header_format)

worksheet.merge_range('C'+str(trow+8)+':F'+str(trow+8),'PEDIDOS  POR COBRAR MXN',blue_header_format)
worksheet.merge_range('C'+str(trow+9)+':F'+str(trow+9),'PEDIDOS POR COBRAR DLL',blue_header_format)
worksheet.merge_range('C'+str(trow+10)+':F'+str(trow+10),'PEDIDOS TOTALES POR COBRAR',blue_header_format)
worksheet.merge_range('C'+str(trow+11)+':F'+str(trow+11),'PEDIDOS TOTALES COBRADOS',blue_header_format)
worksheet.merge_range('C'+str(trow+12)+':F'+str(trow+12),'CLIENTES TOTALES REPORTADOS',blue_header_format)
#volver a traer los datos originales
# pedidos=pd.read_sql("""Select internal_orders.* ,customers.clave,customers.alias,
# coins.exchange_sell, coins.coin, coins.symbol, coins.code
# from ((
#     internal_orders
#     inner join customers on customers.id = internal_orders.customer_id )
#     inner join coins on internal_orders.coin_id = coins.id)
#      """,cnx)
cobros=pd.read_sql("""select cobro_orders.*,internal_orders.coin_id
                     from (((
                         cobro_orders 
    inner join cobros on cobros.id=cobro_orders.cobro_id)
    inner join internal_orders on internal_orders.id = cobros.order_id )
    inner join coins on internal_orders.coin_id = coins.id) """,cnx)
derechos_adquiridos=( pedidos.loc[pedidos['coin_id']==1,'total'].sum() +pedidos.loc[pedidos['coin_id']!=1,'total'].sum()*tc  )/1.16

cobrado=( cobros2.loc[cobros2['coin_pedido']==1,'amount'].sum() +cobros2.loc[cobros2['coin_pedido']!=1,'amount'].sum()*tc  )/1.16
por_cobrar=derechos_adquiridos-cobrado
worksheet.merge_range('G'+str(trow+4)+':H'+str(trow+4),derechos_adquiridos,blue_content_bold)
# worksheet.write_formula('G'+str(trow+4)+':H'+str(trow+4),  '{=(I'+str(trow)+'+J'+str(trow)+' * '+str(tc)+')}',blue_content_bold)
worksheet.merge_range('G'+str(trow+5)+':H'+str(trow+5),cobrado,blue_content_bold)
# worksheet.write_formula('G'+str(trow+5)+':H'+str(trow+5),  '{=(K'+str(trow)+'+L'+str(trow)+' * '+str(tc)+')}',blue_content_bold)

worksheet.merge_range('G'+str(trow+6)+':H'+str(trow+6),por_cobrar,blue_content_bold)
# worksheet.write_formula('G'+str(trow+6)+':H'+str(trow+6),  '{=(M'+str(trow)+'+N'+str(trow)+' * '+str(tc)+')}',blue_content_bold)
pedidos_x_cobrar=0
pedidos_x_cobrar_mx=0
pedidos_x_cobrar_dll=0
for i in range(0,len(pedidos)):
   if(pedidos['coin_id'].values[i]==1):
        if(pedidos['total'].values[i]- cobros.loc[cobros['order_id']==pedidos['id'].values[i],'amount'].sum()-1<=0):
           x=1 
        else:
            pedidos_x_cobrar_mx=pedidos_x_cobrar_mx+1
   else:#osea si no es moneda nacional
        if(pedidos['total'].values[i]- cobros.loc[cobros['order_id']==pedidos['id'].values[i],'amount'].sum()-1<=0):
        #osease, si ya se cobro
           x=4
        else:
            pedidos_x_cobrar_dll=pedidos_x_cobrar_dll+1
            
worksheet.merge_range('G'+str(trow+7)+':H'+str(trow+7),str(len(pedidos)),blue_content_bold)
pedidos_x_cobrar=pedidos_x_cobrar_mx+pedidos_x_cobrar_dll
worksheet.merge_range('G'+str(trow+8)+':H'+str(trow+8),str(pedidos_x_cobrar_mx),blue_content_bold)
worksheet.merge_range('G'+str(trow+9)+':H'+str(trow+9),str(pedidos_x_cobrar_dll),blue_content_bold)
worksheet.merge_range('G'+str(trow+10)+':H'+str(trow+10),str(pedidos_x_cobrar),blue_content_bold)
worksheet.merge_range('G'+str(trow+11)+':H'+str(trow+11),str(len(pedidos)-pedidos_x_cobrar),blue_content_bold)
worksheet.merge_range('G'+str(trow+12)+':H'+str(trow+12),str(len(pedidos['customer_id'].unique())),blue_content_bold)

#Rellenar
worksheet.merge_range('E'+str(trow)+':F'+str(trow+2),' ',blue_header_format)
worksheet.merge_range('O'+str(trow)+':O'+str(trow+2), ' ', blue_header_format)
worksheet.merge_range('T'+str(trow)+':T'+str(trow+2), ' ', blue_header_format)

worksheet.set_column('A:A',15)

worksheet.set_column('D:D',11)
worksheet.set_column('L:L',15)
worksheet.set_column('G:G',15)
worksheet.set_column('H:H',15)
worksheet.set_column('I:N',16)
worksheet.set_column('P:T',15)

worksheet.set_landscape()
worksheet.set_paper(9)
worksheet.fit_to_pages(1, 0)  
workbook.close()


c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site

!!!!!!!!!!!!!!!!!!!!!!!!!!!!
7cliente          8 FEDEX
totalmn 837859.0000000001
xcobrar 153763.00000000012
totaldll 88075.42000000001
xcobrardll 88075.42000000001
!!!!!!!!!!!!!!!!!!!!!!!!!!!!
9cliente         10 TIENDAS 3B
totalmn 3259900.0
xcobrar 340590.0
totaldll 0.0
xcobrardll 0
!!!!!!!!!!!!!!!!!!!!!!!!!!!!
125cliente        127 MI GRANJITA
totalmn 779400.0
xcobrar 86206.89655172406
totaldll 0.0
xcobrardll 0
!!!!!!!!!!!!!!!!!!!!!!!!!!!!
151cliente        156 CRISTALERIAS
totalmn 1127780.5
xcobrar 665873.5
totaldll 0.0
xcobrardll 0
!!!!!!!!!!!!!!!!!!!!!!!!!!!!
152cliente        157 ARTECHE
totalmn 1463800.0
xcobrar 403352.1293103448
totaldll 0.0
xcobrardll 0
!!!!!!!!!!!!!!!!!!!!!!!!!!!!
165cliente        170 AGROQUIMICOS
totalmn 312500.0
xcobrar 6034.482758620696
totaldll 0.0
xcobrardll 0
!!!!!!!!!!!!!!!!!!!!!!!!!!!!
180cliente        185 SERVA
totalmn 127994.45000000001
xcobrar 63997.89827586208
totaldll 0.0
xcobrardll 0
!!!!!!!!!!!!!!!!!!!!!!!!!!!!
220cliente        228 DESSIA
to

c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [34]:
1.7*3

5.1

In [9]:
170*1.5

255.0

In [15]:
facturas['date']

1    09-04-2025
2    04-07-2025
Name: date, dtype: object

In [8]:
hpagos['date']

0    2025-04-02
1    2025-05-13
2    2025-05-13
Name: date, dtype: object

In [4]:
858748.0*2

1717496.0

In [27]:
facturas['estado'].unique()

array(['autorizado', 'CANCELADO'], dtype=object)

In [19]:
print(pd.read_sql(f'select factures.*,cobro_factures.cobro_id cobro_id from factures inner join cobro_factures on cobro_factures.facture_id=factures.id where order_id ={id}',cnx))

    id  order_id  ordinal facture    amount   status npagos  created_at  \
0  881       687        1  A 4176  323327.0  CAPTURA   None  2025-03-06   
1  881       687        1  A 4176  323327.0  CAPTURA   None  2025-03-06   
2  881       687        1  A 4176  323327.0  CAPTURA   None  2025-03-06   
3  881       687        1  A 4176  323327.0  CAPTURA   None  2025-03-06   

   updated_at        date  coin_id   tc  cobro_id  
0  2025-03-06  2025-02-28        1  1.0      1039  
1  2025-03-06  2025-02-28        1  1.0      1078  
2  2025-03-06  2025-02-28        1  1.0      1103  
3  2025-03-06  2025-02-28        1  1.0      1138  


c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [7]:
400/28

14.285714285714286

In [10]:
2200/4

550.0

In [21]:
cobros=pd.read_sql("""Select cobros.* ,
    customers.customer,customers.customer_suburb, customers.clave,
    internal_orders.invoice, internal_orders.payment_conditions,
    internal_orders.category,internal_orders.description,internal_orders.status,
    coins.exchange_sell, coins.coin, coins.symbol,
    banks.bank_description,
    capturistas.name as capturista, revisores.name as revisor, autorizadores.name as autorizador
    from (((((((
    cobros inner join internal_orders on internal_orders.id = cobros.order_id) 
    inner join customers on customers.id = internal_orders.customer_id )
    inner join coins on internal_orders.coin_id = coins.id)
    left join users as capturistas on cobros.capturo=capturistas.id)
    left join users as revisores on cobros.reviso=revisores.id)
    left join users as autorizadores on cobros.autorizo=autorizadores.id))
    inner join banks on banks.id=cobros.bank_id """,cnx)

In [9]:
cobros.loc[cobros['order_id']==681]

NameError: name 'cobros' is not defined

In [7]:
459692.30/2

229846.15

In [20]:
orders.loc[orders['date'].astype(str)>str(date.strftime("%Y"))+'-01-01',"total"]

423    5.183228e+04
424    1.003168e+05
425    1.026716e+04
426    1.295720e+04
427    1.088080e+05
428    1.128912e+05
429    6.275600e+04
430    7.946000e+04
431    6.739600e+04
432    4.609944e+03
433    8.120000e+03
434    3.961400e+05
435    5.707200e+04
436    2.587960e+05
437    8.651280e+03
438    1.063720e+05
439    2.489360e+03
440    1.651724e+06
441    2.683672e+05
442    3.209720e+05
443    3.209720e+05
444    3.430120e+05
445    1.894280e+05
446    6.275832e+04
447    0.000000e+00
448    1.940100e+04
449    4.369720e+04
450    1.263240e+05
451    0.000000e+00
452    2.487040e+05
453    9.326400e+04
454    2.379160e+06
455    0.000000e+00
456    4.717720e+06
457    2.379160e+06
458    4.609944e+03
459    6.496000e+04
460    1.476680e+05
461    0.000000e+00
462    3.781600e+04
463    0.000000e+00
464    0.000000e+00
465    5.707200e+04
466    0.000000e+00
Name: total, dtype: float64

In [16]:
orders.loc[orders['date'].astype(str)>'2025-01-01','total'].sum()

14925724.8488

In [19]:
str(date.strftime("%Y"))+'01-01'

'202501-01'

In [ ]:
import datetime
from dateutil.relativedelta import relativedelta
import numpy as np

year = datetime.date.today().year
quincena=12
month = np.ceil(quincena/ 2)
isFirstHalf = quincena % 2 != 0
startDate =  str(year)+"-"+str(int(month)).zfill(2)+"-01" if isFirstHalf else  str(year)+"-"+str(int(month)).zfill(2)+"-15"
endDate =  str(year)+"-"+str(int(month)).zfill(2)+"-14" if isFirstHalf else  str((datetime.datetime(year,int(month),1 )+relativedelta(months=1))-datetime.timedelta(days=1))[:10];

2025-06-15 2025-06-30


In [ ]:
#siguiente mes en dia primero


'2025-05-31'

In [11]:
'1'.zfill(2)

'01'

In [19]:
facturas.loc[facturas['id'].isin(cobros_facturas.loc[cobros_facturas['cobro_id'].isin(cobros['cobro_id'].values)].facture_id.values)]

,id,order_id,ordinal,facture,amount,status,npagos,created_at,updated_at,date,coin_id,tc
1,826,620,1,A 4115,818738.0,CAPTURA,None,2025-01-10,2025-01-10,2025-01-10,1,1.0
2,835,620,1,A 4125,1309980.0,CAPTURA,None,2025-01-27,2025-01-27,2025-01-21,1,1.0


In [20]:
facturas

,id,order_id,ordinal,facture,amount,status,npagos,created_at,updated_at,date,coin_id,tc
0,821,620,1,A 4110,818738.0,CAPTURA,None,2025-01-03,2025-01-03,2024-12-30,1,1.0
1,826,620,1,A 4115,818738.0,CAPTURA,None,2025-01-10,2025-01-10,2025-01-10,1,1.0
2,835,620,1,A 4125,1309980.0,CAPTURA,None,2025-01-27,2025-01-27,2025-01-21,1,1.0


In [23]:
818738.0+1309980.0

2128718.0

In [42]:
for i in cobros.cobro_id.unique():
    print(i)
    fac_ids=cobros_facturas.loc[cobros_facturas['cobro_id']==i].facture_id.unique()
    for j in fac_ids:
        print('',j)
        print(len(facturas.loc[facturas['id']==j,['id','facture']]))

214
 150
1
 159
1
215
 151
1
216
 152
1
 153
1
217
 154
1
 155
1
249
 157
1
 158
1
 212
1
483
 417
1
 426
0
 440
1
 441
1
 442
0
487
 374
1
 443
1
542
 455
1
573
 515
1
635
 515
1
665
 544
1


In [10]:
cobros.facture_id

AttributeError: 'DataFrame' object has no attribute 'facture_id'

In [32]:
all_facturas

NameError: name 'all_facturas' is not defined

In [18]:
cobros_facturas.loc[cobros_facturas['cobro_id']==679]

,cobro_id,facture_id,id,created_at,updated_at
639,679,167,1034,2025-01-26,2025-01-26


In [26]:
allcobros['ordinal']=range(1,len(allcobros)+1)

C:\Users\hp\AppData\Local\Temp\ipykernel_14000\141872390.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  allcobros['ordinal']=range(1,len(allcobros)+1)


In [27]:
allcobros

,cobro_id,order_id,id,created_at,updated_at,amount,comp,date,bank_id,customer_id,invoice,noha,seller_id,comision,total,ordinal
33,58,155,40,None,None,1252626.0,1862,2022-12-29,2,446,2950,2022,14,0.003,2505252.0,1
34,59,155,41,None,None,1002100.8,1940,2023-03-28,2,446,2950,2022,14,0.003,2505252.0,2
540,595,155,580,2024-03-01,2024-03-01,250525.2,2297,2024-03-01,2,446,2950,2022,14,0.003,2505252.0,3


In [ ]:
cobros[['invoive']]

,cobro_id,order_id,id,created_at,updated_at,amount,comp,date,bank_id,customer_id,invoice,noha,seller_id,comision,total
0,21,116,4,None,None,150000.00,1066,2020-02-24,2,506,2175,12020,2,0.030,356120.00
1,22,116,5,None,None,20000.00,1472,2021-08-16,2,506,2175,12020,2,0.030,356120.00
2,23,116,6,None,None,20000.00,1492,2021-09-17,2,506,2175,12020,2,0.030,356120.00
3,24,116,7,None,None,23200.00,1526,2021-11-12,2,506,2175,12020,2,0.030,356120.00
4,25,116,8,None,None,23200.00,1552,2021-12-23,2,506,2175,12020,2,0.030,356120.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
632,695,332,688,2024-05-17,2024-05-17,55482.68,2381,2024-05-16,1,418,3102,1412023,14,0.003,1090632.00
633,695,451,689,2024-05-17,2024-05-17,92624.84,2381,2024-05-16,1,418,3102,1412023,14,0.003,1090632.00
634,696,438,690,2024-05-17,2024-05-17,444048.00,2382,2024-05-16,2,337,3191,42,5,0.003,1110120.00
635,697,398,691,2024-05-17,2024-05-17,502194.16,2383,2024-05-17,2,424,3160,11,2,0.003,1062388.32
